In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_breast_cancer_df, load_diabetes_df
from sklearn.model_selection import train_test_split
from utils.preprocessing import preprocess_df
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.watcher import  generate_watcher_result, process_result

from utils.save import save_result_as_csv

### Disable TF2 and enable TF1 for alibi.
tf.get_logger().setLevel(40) 
tf.compat.v1.disable_v2_behavior()
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  False


In [2]:
#### Select dataset ####

dataset_name = 'breast_cancer' # [adult, german, compas]

if dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
elif dataset_name == 'breast_cancer':
    dataset_loading_fn = load_breast_cancer_df
else:
    raise Exception("Unsupported dataset")

In [3]:
#### Load datafram info.
df_info = preprocess_df(dataset_loading_fn)

In [4]:
### Seperate to train and test set.
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [5]:
### Get training and testing array.
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [6]:
## Train models.
models = train_three_models(X_train, y_train)

## Save models.
save_three_models(models, dataset_name)

Train on 455 samples
Epoch 1/20
455/455 [==============================] - 0s 158us/sample - loss: 0.6891 - acc: 0.5890
Epoch 2/20
455/455 [==============================] - 0s 11us/sample - loss: 0.6788 - acc: 0.8044
Epoch 3/20
455/455 [==============================] - 0s 13us/sample - loss: 0.6660 - acc: 0.8198
Epoch 4/20
455/455 [==============================] - 0s 12us/sample - loss: 0.6482 - acc: 0.8462
Epoch 5/20
455/455 [==============================] - 0s 14us/sample - loss: 0.6256 - acc: 0.8703
Epoch 6/20
455/455 [==============================] - 0s 16us/sample - loss: 0.5986 - acc: 0.8835
Epoch 7/20
455/455 [==============================] - 0s 19us/sample - loss: 0.5632 - acc: 0.8901
Epoch 8/20
455/455 [==============================] - 0s 22us/sample - loss: 0.5237 - acc: 0.8791
Epoch 9/20
455/455 [==============================] - 0s 14us/sample - loss: 0.4768 - acc: 0.9011
Epoch 10/20
455/455 [==============================] - 0s 12us/sample - loss: 0.4313 - acc: 0.89

In [7]:
### Load models
models = load_three_models(X_train.shape[-1], dataset_name)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [8]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.9737] | RF [0.9912] | NN [0.9737]


## Watcher Counterfactual

In [9]:
### Setting up the CF generating amount.
num_instances = 5
num_cf_per_instance = 1

In [10]:
### Generate CF
results = generate_watcher_result(df_info, train_df, models, num_instances, num_cf_per_instance, X_train, X_test, y_test, max_iters=1000)
result_dfs = process_result(results, df_info)

Finding counterfactual for dt
instance 0
CF 0
Found CF
instance 1
CF 0
Found CF
instance 2
CF 0
CF not found
instance 3
CF 0
Found CF
instance 4
CF 0
CF not found
Finding counterfactual for rfc
instance 0
CF 0
Found CF
instance 1
CF 0
Found CF
instance 2
CF 0
CF not found
instance 3
CF 0
Found CF
instance 4
CF 0
CF not found
Finding counterfactual for nn
instance 0
CF 0
Found CF
instance 1
CF 0
Found CF
instance 2
CF 0
CF not found
instance 3
CF 0
Found CF
instance 4
CF 0
CF not found


In [11]:
### Save result as file.
save_result_as_csv("watcher", dataset_name, result_dfs)

Result has been saved to ./results/watcher_breast_cancer
